In [1]:


import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from matplotlib import pyplot as plt
import gc
import re
import warnings
warnings.filterwarnings('ignore')
pd.set_option.max_columns = None
pd.set_option.max_rows = None
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer



In [9]:
test = pd.read_csv(r'C:\projects\Python\dmia\Sport\test1\kaggel\test.csv')
train = pd.read_csv(r'C:\projects\Python\dmia\Sport\test1\kaggel\train.csv')



In [3]:
train.head()

,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


### Замен ответов не стоит делать так как в тестово выборке могут быть такие же ошибки!

### 1.Вижу что все слова с маленькой буквы идут с ошибкой меняю у них Label на 0

In [196]:
lower_error = train[(train['Word'].str.contains('^[а-я]')) & (train['Label'] ==1)]['Label']

In [197]:
index = lower_error.index.values

In [198]:
train['Label'].loc[index] = train['Label'].loc[index].replace(1,0)

In [199]:
train['Label'].value_counts()

0    92036
1     9372
Name: Label, dtype: int64

### Теперь можно брать ответы!

In [10]:
y_train = train['Label']
y_train.shape

(101408,)

### 1.1 Добавляю новые натуральные признаки 

In [16]:
Full_train_dop_fet = pd.DataFrame(index = Full_train.index)

Full_train['Upper'] = Full_train['Word'].apply(lambda x: 1 if (x[0].isupper() and x[1:].islower()) else 0)

Full_train['Lower'] = Full_train['Word'].apply(lambda x: 1 if (x.islower()) else 0)

Full_train['All_upper'] = Full_train['Word'].apply(lambda x: 1 if (x.isupper()) else 0)

Full_train['Length'] = Full_train['Word'].apply(lambda x: len(x))

Full_train['Vow'] = Full_train['Word'].apply(lambda x: len(re.findall('[уеыаоэяию]', x, re.IGNORECASE)))

Full_train['Con'] = Full_train['Length'] - Full_train['Vow'] 


### 2. Соединяю 2 выборки

In [11]:
train = train.drop('Label', axis = 1)

In [12]:
idx = train.shape[0]

In [13]:
Full_train = pd.concat([train, test])

In [14]:
Full_train.shape,train.shape,y_train.shape

((290328, 1), (101408, 1), (101408,))

In [17]:
Full_train

,Word,Upper,Lower,All_upper,Length,Vow,Con
0,Аалтонен,1,0,0,8,4,4
1,Аар,1,0,0,3,2,1
2,Аарон,1,0,0,5,3,2
3,ААРОН,0,0,1,5,3,2
4,Аарона,1,0,0,6,4,2
5,Аарона,1,0,0,6,4,2
6,Аароне,1,0,0,6,4,2
7,Ааронов,1,0,0,7,4,3
8,Аахена,1,0,0,6,4,2
9,Абабков,1,0,0,7,3,4


### 2.1 Пробую сделать выборку без лейбл кодировки

In [259]:
Full_df = Full_train.drop('Word', axis = 1)

### 3. Применяю LabelCodirovanie, Standart Scaller

In [209]:
Lb = LabelEncoder()

In [210]:
Full_train['Word'] = Lb.fit_transform(Full_train['Word'])

In [211]:
scr = StandardScaler()

In [212]:
Full_train['Word'] = scr.fit_transform(Full_train['Word'].values.reshape(-1,1))

### 4. Добавляю полиномы от этих чисел в качестве признаков.


In [254]:
poly = PolynomialFeatures(degree=7)
X_poly = poly.fit_transform(Full_train['Word'])
X_poly[:idx].shape

ValueError: Expected 2D array, got 1D array instead:
array=[-1.71936325 -1.71935107 -1.71931452 ..., -1.73316692  1.72982213
  1.72983432].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

### 5. Применяю логистичческую регрессию!

In [154]:
log_reg = LogisticRegression()

In [155]:
log_reg.fit(X_poly[:idx], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [160]:
y_pred = log_reg.predict_proba(X_poly[:idx])[:,1]

In [161]:
score = roc_auc_score(y_train, y_pred)
score

0.82355964988518748

### 5.1 С новыми признаками

In [237]:
c_values = np.logspace(0, 15, 3)

In [238]:
Log_reg_new_feat = LogisticRegressionCV(Cs = c_values, cv= 3, random_state=17, n_jobs=-1)

In [239]:
Log_reg_new_feat.fit(Full_train[:idx], y_train)

LogisticRegressionCV(Cs=array([  1.00000e+00,   3.16228e+07,   1.00000e+15]),
           class_weight=None, cv=3, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='ovr',
           n_jobs=-1, penalty='l2', random_state=17, refit=True,
           scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [240]:
Log_reg_new_feat.C_

array([ 31622776.60168379])

In [241]:
prdict_train = Log_reg_new_feat.predict_proba(Full_train[:idx])[:,1]

In [242]:
score = roc_auc_score(y_train,prdict_train)
score

0.872025926474576

In [227]:
log = LogisticRegression()

In [228]:
log.fit(Full_train[:idx], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [229]:
prdict = log.predict_proba(Full_train[:idx])[:,1]

In [230]:
score = roc_auc_score(y_train,prdict)
score

0.87206657285676437

### 5.2 Без Word

In [266]:
lr3 = LogisticRegressionCV(Cs = c_values, cv= 3, random_state=17, n_jobs=-1)

In [268]:
lr3.fit(Full_df[:idx],y_train)

LogisticRegressionCV(Cs=array([  1.00000e+00,   3.16228e+07,   1.00000e+15]),
           class_weight=None, cv=3, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='ovr',
           n_jobs=-1, penalty='l2', random_state=17, refit=True,
           scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [269]:
y_pred_regul = lr3.predict_proba(Full_df[:idx])[:, 1]
# считаем качество
score = roc_auc_score(y_train, y_pred_regul)
score

0.86989010690615287

### 6. Применяю кросс валидацию и подбор регуляризации

In [172]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [173]:
c_values = np.logspace(-10, 5, 10)

logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, verbose=1, n_jobs=-1)
logit_searcher.fit(X_poly[:idx], y_train)

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.4s finished


LogisticRegressionCV(Cs=array([  1.00000e-10,   4.64159e-09,   2.15443e-07,   1.00000e-05,
         4.64159e-04,   2.15443e-02,   1.00000e+00,   4.64159e+01,
         2.15443e+03,   1.00000e+05]),
           class_weight=None,
           cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='ovr', n_jobs=-1, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=1)

In [174]:
logit_searcher.C_

array([  1.00000000e-10])

In [175]:
y_pred_C = logit_searcher.predict_proba(X_poly[:idx])[:,1]

In [176]:
score = roc_auc_score(y_train, y_pred_C)
score

0.81997554325965005

### 7. Применяю случайный лес.

In [178]:
first_tree = RandomForestClassifier()

In [256]:
tree_params = {'max_depth': [3, 5, 7, 9], 
               'min_samples_leaf': list(range(5, 10)),
               'max_features': ['log2', 'sqrt','auto'],
               'n_estimators': [1, 3, 5, 10]}

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [257]:
locally_best_tree = GridSearchCV(first_tree, tree_params, cv=3, n_jobs= -1)
locally_best_tree.fit(Full_train[:idx], y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [5, 6, 7, 8, 9], 'max_features': ['log2', 'sqrt', 'auto'], 'n_estimators': [1, 3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [258]:
locally_best_tree.best_params_, locally_best_tree.best_score_

({'max_depth': 3,
  'max_features': 'log2',
  'min_samples_leaf': 6,
  'n_estimators': 1},
 0.90758125591669292)

In [253]:
y_pred_tree3 = locally_best_tree.predict_proba(Full_train[idx:])[:,1]

In [255]:
Send  = pd.Series(y_pred_tree3, index=range(0, y_pred_tree3.shape[0]),
         name='Prediction').to_csv('randomtree3.csv', header=True, index_label='id')

### Без ЛБ кодировки

In [270]:
locally_tree = GridSearchCV(first_tree, tree_params, cv=3, n_jobs= -1)
locally_tree.fit(Full_df[:idx], y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [5, 6, 7, 8, 9], 'max_features': ['log2', 'sqrt', 'auto'], 'n_estimators': [1, 3, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [273]:
locally_tree.best_params_, locally_tree.best_score_

({'max_depth': 3,
  'max_features': 'log2',
  'min_samples_leaf': 5,
  'n_estimators': 1},
 0.90758125591669292)

In [274]:
y_pred_tree4 = locally_tree.predict_proba(Full_df[idx:])[:,1]

In [275]:
Send  = pd.Series(y_pred_tree4, index=range(0, y_pred_tree4.shape[0]),
         name='Prediction').to_csv('randomtree4.csv', header=True, index_label='id')

### Пробую использовать Векторизатор 

In [63]:
def select_bin(data):
    return data[:, binary_data_indices]

def select_num(data):
    return data[:, numeric_data_indices]

def select_cat(data):
    return data[:, categorical_data_indices]

def select_sparse(data):
    return data[:, sparse_numeric_data_indices]